In [1]:
import pandas as pd
import numpy as np
# import seaborn as sn
import matplotlib.pyplot as plt
import itertools

dx_use_final = pd.read_csv("DX_USE.csv")


def vc2mon(viscode):

    viscode = np.array(viscode)
    month = np.zeros_like(viscode, dtype=np.int32)
    for i, vc in enumerate(viscode):
        if vc == 'nv':
            month[i] = -1
        elif vc != 'bl' and vc != 'sc' and vc != 'scmri':
            month[i] = int(vc[1:])

    return month


In [2]:
dx_use = dx_use_final
print(dx_use['VISCODE2'].unique())
dx_use['Month'] = vc2mon(dx_use['VISCODE2'])


['bl' 'm06' 'm12' 'm24' 'm18' 'm36' 'm48' 'm60' 'm72' 'sc' 'm84' 'm96'
 'm108' 'm120' 'm132' 'm144' 'm156' 'm168' 'm180']


In [3]:
data = pd.read_csv("MRI_BAI_features.csv")
mri_data = data[['RID', 'VISCODE2']]
mri_data['Month'] = vc2mon(mri_data['VISCODE2'])

C:\Users\zhengzhiyang\Anaconda3\envs\ml\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [34]:
def get_feature_dx(dx_use, features):
    feature_pet = pd.merge(
        features.drop(['VISCODE2'], axis=1),
        dx_use[['RID', 'VISCODE2', 'Month', 'DX']], how='inner', on=['RID'])

    feature_pet.drop_duplicates(inplace=True)
    is_feature = np.zeros((len(feature_pet),), dtype=np.int32)
    is_feature[feature_pet['Month_x'] == feature_pet['Month_y']] = 1
    feature_pet.loc[:, ['is_feature']] = is_feature

    # Delete the replicated rows
    feature_pet_1 = pd.merge(feature_pet.drop(['Month_x'], axis=1), feature_pet.drop(
        ['Month_x'], axis=1), how='inner', on=['RID', 'Month_y', 'DX'])
    feature_pet_1['is_feature'] = np.maximum(
        feature_pet_1['is_feature_x'], feature_pet_1['is_feature_y'])
    feature_pet_1.drop(['is_feature_x', 'is_feature_y'], axis=1, inplace=True)
    feature_pet_1.sort_values(
        by=['RID', 'Month_y', 'is_feature'], inplace=True)
    feature_pet_1.drop_duplicates(
        subset=['RID', 'Month_y'], keep='last', inplace=True)
    feature_pet_1.drop(['VISCODE2_y'], axis=1, inplace=True)

    return feature_pet_1


In [35]:
feature_mri = get_feature_dx(dx_use, mri_data)
feature_mri.rename(columns={"Month_y": "Month", "VISCODE2_x": "VISCODE2"}, inplace=True)
feature_mri.head()

,RID,VISCODE2,Month,DX,is_feature
135276,33,bl,0,2,1
135310,33,m06,6,2,1
135350,33,m12,12,2,1
135383,33,m18,18,2,1
135421,33,m24,24,2,1


In [41]:
data = pd.read_csv("MRI_BAI_features.csv")
data['Month'] = vc2mon(data['VISCODE2'])
feature_mri_1 = pd.merge(
    feature_mri, data, how='outer', on=['RID', 'Month'],
    suffixes=('_dx', '_mri'))


In [42]:
feature_mri_1.to_csv("MRI_features_all.csv", index=False)